In [2]:
import pandas as pd
import ast
#f = open("./busstopv2.txt", "r")
#pd.read_csv(pd.compat.StringIO("\n".join(lines)), sep=";")

In [3]:
with open("busstopv2.txt", "r") as grilled_cheese:
    lines = grilled_cheese.readlines()
    i=0
    collector = []
    for item in lines:
        if i < 1000:
            collector.append(ast.literal_eval(item))
            #df = dict(item)
            #print(df)
            #print(type(df))
            i+=1
    print(len(collector))   
    #dictlines = dict(lines)
    #print(dictlines["value"])

11


In [29]:
i=2
df = pd.DataFrame(collector[i]['value'],index=[i for i in range(len(collector[1]['value']))])
print(df)

    BusStopCode        RoadName        Description  Latitude   Longitude
0         22161  Pioneer Rd Nth   Aft Lok Yang Way  1.325517  103.696431
1         22169  Pioneer Rd Nth        Aft Intl Rd  1.326245  103.696616
2         22171  Pioneer Rd Nth  Opp Secura S'pore  1.329542  103.696377
3         22179  Pioneer Rd Nth      Secura S'Pore  1.330005  103.696659
4         22181  Pioneer Rd Nth        S'pore Post  1.333016  103.696377
..          ...             ...                ...       ...         ...
495       28071    Boon Lay Way            Blk 105  1.337650  103.739042
496       28079    Boon Lay Way            Blk 210  1.338399  103.738601
497       28081    Boon Lay Way            Blk 112  1.339508  103.736648
498       28089    Boon Lay Way       Bef Yuhua CC  1.340164  103.736346
499       28091    Boon Lay Way       Lakeside Stn  1.344258  103.720125

[500 rows x 5 columns]


In [33]:
import openpyxl
dfs = []
for i in range(len(collector)):
    dfs.append(pd.DataFrame(collector[i]['value'],index=[i for i in range(len(collector[i]['value']))]))
print(f"Done with appending {i} DataFrames")
for i in range(len(collector)):
    #pass
    print(f"Dataframe {i+1}:\n")
    print(dfs[i])
    
overall = pd.concat(dfs,axis=0)
print("Overall DataFrame:")
print(overall.shape)


Done with appending 10 DataFrames
Dataframe 1:

    BusStopCode        RoadName                Description  Latitude  \
0         01012     Victoria St        Hotel Grand Pacific  1.296848   
1         01013     Victoria St            St. Joseph's Ch  1.297710   
2         01019     Victoria St            Bras Basah Cplx  1.296990   
3         01029   Nth Bridge Rd               Opp Natl Lib  1.296673   
4         01039   Nth Bridge Rd                 Bugis Cube  1.298208   
..          ...             ...                        ...       ...   
495       14019     Kg Bahru Rd         Opp Mt Faber Lodge  1.268489   
496       14021  Lower Delta Rd       The Pearl @ Mt Faber  1.271330   
497       14029  Lower Delta Rd                    Blk 105  1.272473   
498       14031  Lower Delta Rd      St. Theresa's Convent  1.276243   
499       14039  Lower Delta Rd  Opp St. Theresa's Convent  1.276636   

      Longitude  
0    103.852536  
1    103.853225  
2    103.853022  
3    103.854414

### Now that DataFrame is successfully created, output into Excel spreadsheet:

In [35]:
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
wb = Workbook()
ws = wb.active

# For each row in DataFrame
for r in dataframe_to_rows(overall, index=True, header=True):
    ws.append(r)

for cell in ws['A'] + ws[1]:
    cell.style = 'Pandas'

wb.save("busstop_overall.xlsx")

### Proceed to compare distance from HDB to each bus stop.
Load all HDB data first:

In [45]:
allhdb = pd.read_excel("./hdb_to_mrt_all.xlsx", index_col=0,engine='openpyxl') #import fresh data  

In [88]:
# Find relevant long and lat for HDB (do not mix up with MRT)
allhdb.columns
global blocks
blocks = allhdb[['postal','lng_hdb','lat_hdb']].drop_duplicates()
#blocks = blocks[blocks['postal']=='650383']
print(blocks)


       postal     lng_hdb   lat_hdb
26103  650383  103.752217  1.362323


In [136]:
def distance(long1,lat1,long2,lat2):
    # 1 deg = 111000m
    dist = 111000*((long2-long1)**2 +(lat2-lat1)**2)**0.5
    return dist

def all_distance(flist, overall):
    '''Returns closest bus stop from each HDB block in Singapore'''
    # For each block
    all_dist = {}
    min_dist = {}
    alldist_df = pd.DataFrame()

    margin_h = 0.004
    margin_v = 0.004  # of a degree. 1 degree = 111000m
    # 0.004 = 444m region
    box ={}
    # flight - index
    # fdata - block's long and lat
    for flight, fdata in flist.iterrows():
        #print(f"fdata is:\n{fdata}")
        #print(f"{fdata.shape}")
        print(f"Working on {fdata['postal']}")
        long1 = fdata['lng_hdb']
        lat1 = fdata['lat_hdb']
        # Compare row-by-row with all bus stops:
        print(type(long1),type(lat1))
        box[flight]=[]
        for row, busstop in overall.iterrows():
            #print(busstop)
            long2 = busstop['Longitude']
            lat2 = busstop['Latitude']
            #print(f"Long2 is {long2}, Lat2 is {lat2}")
            #print(type(long2),type(lat2))
            if abs(long2-long1) > margin_h:
                continue
            elif abs(lat2-lat1) > margin_v:
                continue
            else:
                busstop['Distance']  = distance(long1,lat1,long2,lat2)
                #print(f"Bus stop {busstop['Description']}: {busstop['Distance']}m")
                box[flight].append(pd.DataFrame(busstop))
            #print(f"Distance is {dist}m")

            #dist = math.sqrt(ellipdist(long1, lat1, long, lat)**2)
            #rowdata.insert(loc=0, column='Distance (ft)', value=dist_ft)
        for key, value in box.items():
            #print("The value is",value)
            min_dist[key]= pd.concat(value,axis=1)
            min_dist[key].dropna(inplace=True)
            min_dist[key]=min_dist[key].T
            min_dist[key]['Distance'] = min_dist[key]['Distance'].astype(float)
            min_dist[key] = min_dist[key].nsmallest(2,'Distance')
            print(min_dist[key])
    return min_dist
        
boxes = all_distance(blocks,overall)
print(boxes)
#col = pd.concat(boxes,axis=1)


Working on 650383
<class 'float'> <class 'float'>
    BusStopCode             RoadName  Description  Latitude   Longitude  \
458       43589  Bt Batok West Ave 5  Opp Blk 347  1.362273  103.751532   
457       43581  Bt Batok West Ave 5      Blk 347  1.362519  103.751384   

      Distance  
458  76.282252  
457  95.032670  
{26103:     BusStopCode             RoadName  Description  Latitude   Longitude  \
458       43589  Bt Batok West Ave 5  Opp Blk 347  1.362273  103.751532   
457       43581  Bt Batok West Ave 5      Blk 347  1.362519  103.751384   

      Distance  
458  76.282252  
457  95.032670  }


In [137]:
#boxes = all_distance(blocks,overall)
print(boxes.head(5))

AttributeError: 'dict' object has no attribute 'head'

In [ ]:
print